In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pathlib import Path
import tqdm

base_path = ("/kaggle/input/mlb-player-digital-engagement-forecasting/")
path = ("/kaggle/input/mlb-unnested/")
path2 = ("/kaggle/input/mlb-pdef-train-dataset/")

# players = pd.read_pickle(path + "players.pickle")
# teams   = pd.read_pickle(path + "teams.pickle")
# seasons = pd.read_pickle(path + "seasons.pickle")
# awards  = pd.read_pickle(path + "awards.pickle")
# sample_sub = pd.read_pickle(path + "example_sample_submission.pickle")

players = pd.read_csv(base_path + "players.csv")
teams   = pd.read_csv(base_path + "teams.csv")
seasons = pd.read_csv(base_path + "seasons.csv")
awards  = pd.read_csv(base_path + "awards.csv")
sample_sub = pd.read_csv(base_path + "example_sample_submission.csv")

# train_awards = pd.read_pickle(path + "train_awards.pickle")
# train_events = pd.read_pickle(path + "train_events.pickle")
# train_games  = pd.read_pickle(path + "train_games.pickle")
# train_rosters = pd.read_pickle(path + "train_rosters.pickle")
# train_standings = pd.read_pickle(path + "train_standings.pickle")
# train_transactions = pd.read_pickle(path + "train_transactions.pickle")
# train_engagements  = pd.read_pickle(path + "train_nextDayPlayerEngagement.pickle")
# train_playerboxscores = pd.read_pickle(path + "train_playerBoxScores.pickle")
# train_playertwitters  = pd.read_pickle(path + "train_playerTwitterFollowers.pickle")
# train_teamboxscores = pd.read_pickle(path + "train_teamBoxScores.pickle")
# train_teamtwitters  = pd.read_pickle(path + "train_teamTwitterFollowers.pickle")

train_awards = pd.read_pickle(path2 + "awards_train.pkl")
train_events = pd.read_pickle(path2 + "events_train.pkl")
train_games  = pd.read_pickle(path2 + "games_train.pkl")
train_rosters = pd.read_pickle(path2 + "rosters_train.pkl")
train_standings = pd.read_pickle(path2 + "standings_train.pkl")
train_transactions = pd.read_pickle(path2 + "transactions_train.pkl")
train_engagements  = pd.read_pickle(path2 + "nextDayPlayerEngagement_train.pkl")
train_playerboxscores = pd.read_pickle(path2 + "playerBoxScores_train.pkl")
train_playertwitters  = pd.read_pickle(path2 + "playerTwitterFollowers_train.pkl")
train_teamboxscores = pd.read_pickle(path2 + "teamBoxScores_train.pkl")
train_teamtwitters  = pd.read_pickle(path2 + "teamTwitterFollowers_train.pkl")

# test_awards = pd.read_pickle(path + "example_test_awards.pickle")
# test_events = pd.read_pickle(path + "example_test_events.pickle")
# test_games  = pd.read_pickle(path + "example_test_games.pickle")
# test_rosters = pd.read_pickle(path + "example_test_rosters.pickle")
# test_standings = pd.read_pickle(path + "example_test_standings.pickle")
# test_transactions = pd.read_pickle(path + "example_test_transactions.pickle")
# test_playerboxscores = pd.read_pickle(path + "example_test_playerBoxScores.pickle")
# test_teamboxscores = pd.read_pickle(path + "example_test_teamBoxScores.pickle")
players

In [ ]:
train_events

In [ ]:
train_awards[train_awards["playerId"] == 598264]

In [ ]:
train_engagements.describe()

In [ ]:
train_games

In [ ]:
train_rosters

In [ ]:
train_standings

In [ ]:
train_transactions

In [ ]:
train_teamtwitters

In [ ]:
train_playertwitters

In [ ]:
"""
train_playertwitters["date_new"] = train_playertwitters["date"].dt.strftime("%Y%m%d").astype(int)
train_teamtwitters["date_new"] = train_teamtwitters["date"].dt.strftime("%Y%m%d").astype(int)
"""

In [ ]:
"""
cnt = 0
cnt2 = 0
for i in range(train_playertwitters.shape[0]):
    if(train_playertwitters.loc[i,"dailyDataDate"] == train_playertwitters.loc[i,"date_new"]):
        cnt += 1

for i in range(train_teamtwitters.shape[0]):
    if(train_teamtwitters.loc[i,"dailyDataDate"] == train_teamtwitters.loc[i,"date_new"]):
        cnt2 += 1
print(cnt) # equal to player twitter data length -> not need to date
print(cnt2) # equal to team twitter data length -> not need to date
"""

In [ ]:
train_teamboxscores

In [ ]:
a = train_playerboxscores
b = train_playerboxscores.drop_duplicates(["playerId","gameDate"])
print(a.shape,b.shape)

In [ ]:
train_engagements

In [ ]:
# merge train_rosters
train1 = pd.merge(train_engagements.drop("index",axis=1),train_rosters.drop(["index","status","gameDate"],axis=1),on=["date","playerId"],how="left")
train1

In [ ]:
# merge twitter data
train2_1 = pd.merge(train1,train_playertwitters.drop(["index","playerName","accountName","twitterHandle"],axis=1),on=["date","playerId"],how="left")
train2 = pd.merge(train2_1,train_teamtwitters.drop(["index","teamName","accountName","twitterHandle"],axis=1),on=["date","teamId"],how="left")
train2

In [ ]:
train2 = train2.rename(columns={"numberOfFollowers_x":"player_followers","numberOfFollowers_y":"team_followers"})
train2

In [ ]:
train2.isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train2["statusCode"].values.reshape((-1,1)))
train2["statusCode"] = le.transform(train2["statusCode"].values.reshape((-1,1)))
train2

In [ ]:
train2.groupby(["date","playerId"])["statusCode"].nunique().sort_values()

In [ ]:
from sklearn.model_selection import KFold,GroupKFold
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
# from sklearn.externals import joblib
import pickle

X = train2.drop(["date","engagementMetricsDate","target1","target2","target3","target4"],axis=1)
y = train2.loc[:,["target1","target2","target3","target4"]]

lgb_params = { 'objective': 'mean_absolute_error',
               'boosting_type': 'gbdt',
               'n_estimators': 10,
               'learning_rate': 0.01,
               'num_leaves': 90,
               'colsample_bytree': 0.4,
               'subsample': 0.6,
               'subsample_freq': 2,
               'bagging_seed': 42,
               'reg_alpha': 8,
               'reg_lambda': 2,
               'random_state': 42,
               'n_jobs': -1
             }

# oof = np.zeros((train2.shape[0],4))
# predict = np.zeros((test.shape[0],))
N_SPLITS = 10

valid_predict = []
valid_idxes = []
per_mae = []
models = []

# lgbm = LGBMRegressor(**lgb_params)
# lgbm_model = MultiOutputRegressor(lgbm)
kf = KFold(n_splits=N_SPLITS,shuffle=True,random_state=42)
for i in range(4):
    for fn,(tr_idx,val_idx) in enumerate(kf.split(X,y)):
        train_x,valid_x = X.iloc[tr_idx],X.iloc[val_idx]
        train_y,valid_y = y.iloc[tr_idx,-4+i],y.iloc[val_idx,-4+i]

        lgbm = LGBMRegressor(**lgb_params)
        lgbm.fit(train_x,train_y,eval_set=[(valid_x,valid_y)],early_stopping_rounds=30,verbose=0)
    #     pre_test_lati = lr_lati.predict(test.drop(["prev_lat","prev_lng"],axis=1))
    #     pre_test_long = lr_long.predict(test.drop(["prev_lat","prev_lng"],axis=1))
    #     predict_lati += pre_test_lati / N_SPLITS
    #     predict_long += pre_test_long / N_SPLITS
        
        models.append(lgbm)
#         joblib.dump(lgbm,f"target{i+1} - Fold{fn+1}.learn")
        pre = lgbm.predict(valid_x)
#         val_pre = np.array(oof[val_idx])
#         valid_predict.append(oof[val_idx])
        valid_idxes.append(val_idx)
#         with open(f"target{i+1} - Fold{fn+1}.pickle", mode='wb') as fp:
#             pickle.dump(lgbm,fp)
        
        
        print("Fold",str(fn+1),"：",str(mean_absolute_error(valid_y,pre)))
#         print("Fold",str(fn+1),"：",str(mean_absolute_error(valid_y,pre2)))
        per_mae.append(mean_absolute_error(valid_y,pre))
    print("target",str(i+1),"MAE：",np.mean(per_mae[i*10:(i+1)*10]))
print("Mean Score：",np.mean(per_mae))

In [ ]:
models

In [ ]:
X

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
example_test = pd.read_csv("/kaggle/input/mlb-player-digital-engagement-forecasting/example_test.csv")
unpack_json(example_test["rosters"].iloc[4])

In [ ]:
train_rosters.columns

In [ ]:
import mlb
import json
from tqdm.auto import tqdm
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in tqdm(iter_test):
#     print(test_df.shape)
#     print(sample_prediction_df.shape)
    sample_prediction_df = sample_prediction_df.reset_index()
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId'].map(lambda x: int(x.split('_')[1]))
#     test_rosters = test_df['rosters'].fillna('[]').apply(lambda x:json.loads(x))
#     test_rosters = list(np.concatenate(test_rosters.values))
#     test_rosters = pd.DataFrame(test_rosters)
#     print(test_rosters)

     # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
#         test_rosters['gameDate'] = test_rosters.gameDate.apply(lambda x:x.replace('-','')).astype(int)
    else:
        test_rosters = pd.DataFrame({'playerId':sample_prediction_df['playerId']})
        for col in train_rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
#     test_rosters = test_rosters.rename(columns={'gameDate':'date'})
    
    if test_df['playerTwitterFollowers'].iloc[0] == test_df['playerTwitterFollowers'].iloc[0]:
        test_playertwitters = pd.DataFrame(eval(test_df['playerTwitterFollowers'].iloc[0]))
#         test_rosters['gameDate'] = test_rosters.gameDate.apply(lambda x:x.replace('-','')).astype(int)
    else:
        test_playertwitters = pd.DataFrame({'playerId':sample_prediction_df['playerId']})
        for col in train_playertwitters.columns:
            if col == 'playerId': continue
            test_playertwitters[col] = np.nan
    
    test_playertwitters = test_playertwitters.rename(columns={"numberOfFollowers":"player_followers"})
    
    test = sample_prediction_df[['playerId']].copy()
    df = pd.merge(test,test_rosters[["playerId","teamId","statusCode"]],on="playerId",how='left')
    df = pd.merge(df,test_playertwitters[["playerId","player_followers"]],on="playerId",how="left")
    
    if test_df['teamTwitterFollowers'].iloc[0] == test_df['teamTwitterFollowers'].iloc[0]:
        test_teamtwitters = pd.DataFrame(eval(test_df['teamTwitterFollowers'].iloc[0]))
        test_teamtwitters = test_teamtwitters.rename(columns={"numberOfFollowers":"team_followers"})
        df = pd.merge(df,test_teamtwitters[["teamId","team_followers"]],on="teamId",how="left")
#         test_rosters['gameDate'] = test_rosters.gameDate.apply(lambda x:x.replace('-','')).astype(int)
    else:
        test_teamtwitters = pd.DataFrame({'playerId':sample_prediction_df['playerId']})
        test_teamtwitters["team_followers"] = np.nan
        df = pd.concat([df,test_teamtwitters["team_followers"]],axis=1)
#         for col in train_teamtwitters.columns[1:]:
#             if col == 'pId': continue
#             test_teamtwitters[col] = np.nan
    
#     test = sample_prediction_df[['playerId']].copy()
#     print(test)
#     df = pd.merge(test,test_rosters[["playerId","teamId","statusCode"]],on="playerId",how='left')
#     df = pd.merge(df,test_playertwitters[["playerId","numberOfFollowers"]],on="playerId",how="left")
#     df = pd.merge(df,test_teamtwitters[["teamId","numberOfFollowers"]],on="teamId",how="left")
#     first_day = [20210501,20210601,20210701,20210801,20210901]
#     df["player_followers"] = np.nan
#     df["team_followers"] = np.nan
    
#     predicted = pd.DataFrame(index=df.date)

    df["statusCode"] = le.transform(df["statusCode"].values.reshape(-1,1))
    predict1,predict2,predict3,predict4 = np.zeros((df.shape[0],)),np.zeros((df.shape[0],)),np.zeros((df.shape[0],)),np.zeros((df.shape[0],))
    predict = [predict1,predict2,predict3,predict4]
    targets = ["target1","target2","target3","target4"]
    for i in range(4):
        for j in range(10):
#             with open(f"target{i+1} - Fold{j+1}.pickle", mode='rb') as fp:
#                 lgbm = pickle.load(fp)
#                 pre_test = lgbm.predict(df.drop("date_playerId",axis=1))
#                 predict[i] += pre_test / 10
            model = models[i*10+j]
#             print(i*10+j)
            pre_test = model.predict(df)
            predict[i] += pre_test / 10
        pre = np.clip(predict[i],0,100)
#         print(predict[i])
#         print(len(predict[i]))
        sample_prediction_df[targets[i]] = pre
#         print(df)
    
    sample_prediction_df = sample_prediction_df.fillna(0.)
    sample_prediction_df = sample_prediction_df.set_index("date")
    del sample_prediction_df['playerId']
#     print(sample_prediction_df)
    
#     for i in range(len(predict)):
#         predict[i] = pd.DataFrame(predict[i],columns=[f"target{i+1}"])
#         predicted[f"target{i+1}"] = predict[i]
    
#     predicted['date_playerId'] = sample_prediction_df['date_playerId'].values
#     predicted = predicted.iloc[:,[4,0,1,2,3]]
#     predicted.index = sample_prediction_df.reset_index().date
#     predicted.columns = sample_prediction_df.columns
#     predicted = predicted.iloc[:,[4,0,1,2,3]]
#     print(predicted)
#     print(predicted.shape,sample_prediction_df.shape)
#     assert df.shape[0]==sample_prediction_df.shape[0]

    env.predict(sample_prediction_df)


In [ ]:
df

In [ ]:
i*10+j

In [ ]:
example_test = pd.read_csv("/kaggle/input/mlb-player-digital-engagement-forecasting/example_test.csv")
example_test.loc[0,"rosters"]

In [ ]:
b = [0.2,-1,128,45.3,21]
print(np.clip(b,0,100))

In [ ]:
test_df.loc[:,"rosters"]

In [ ]:
sample_prediction_df

In [ ]:
sample_prediction_df.info()

In [ ]:
sample_sub.info()

In [ ]:
sample_prediction_df

In [ ]:
sample_sub